In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Housing.csv')

# Display the first 5 rows of the DataFrame
#print(df.head())

In [ ]:
from IPython.display import display
display(df)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [ ]:
# List of variables to map

varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

# Defining the map function
def binary_map(x):
    return x.map({'yes': 1, 'no': 0})

# Applying the function to the housing list
df[varlist] = df[varlist].apply(binary_map)
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom',
       'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'price']

df[num_vars] = scaler.fit_transform(df[num_vars])
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,1.000000,0.396564,0.6,0.333333,0.666667,1.0,0.0,0.0,0.0,1.0,0.666667,1.0,furnished
1,0.909091,0.502405,0.6,1.000000,1.000000,1.0,0.0,0.0,0.0,1.0,1.000000,0.0,furnished
2,0.909091,0.571134,0.4,0.333333,0.333333,1.0,0.0,1.0,0.0,0.0,0.666667,1.0,semi-furnished
3,0.906061,0.402062,0.6,0.333333,0.333333,1.0,0.0,1.0,0.0,1.0,1.000000,1.0,furnished
4,0.836364,0.396564,0.6,0.000000,0.333333,1.0,1.0,1.0,0.0,1.0,0.666667,0.0,furnished


In [ ]:
input = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom',
       'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea']

#print(df.columns[1])
X = df.values[:, 1:12]  # get input values from the first column -- X is a list here which is a 1 dimentional array
y = df.values[:, 0]  # get output values from the second column --  Y is a list here which is a 2 dimentional array

X = torch.from_numpy(X.astype(np.float64))
y = torch.from_numpy(y.astype(np.float64))

split = int(0.8*545)
shuffled = torch.randperm(545)
train = shuffled[:split]
val = shuffled[split:]

X_train = X[train]
y_train = y[train]
X_val = X[val]
y_val = y[val]

In [ ]:
weights = np.ones(12)
np.append(weights, 0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.])

In [ ]:
from typing import Any
import torch.nn as nn

linear_model = nn.Linear(in_features=11, out_features=1).double()
#linear_model(X_val)

In [ ]:
linear_model.weight

Parameter containing:
tensor([[ 0.2029,  0.2026,  0.1712, -0.1282, -0.1224, -0.0701, -0.1418, -0.2496,
         -0.1241, -0.2011,  0.2643]], dtype=torch.float64, requires_grad=True)

In [ ]:
linear_model.bias

Parameter containing:
tensor([-0.2624], dtype=torch.float64, requires_grad=True)

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [ ]:
import torch.optim as optim

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

#linear_model = nn.Linear(1, 1) # <1>
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = linear_model,
    loss_fn = loss_fn,
    t_u_train = X_train,
    t_u_val = X_val,
    t_c_train = y_train,
    t_c_val = y_val)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 0.4384, Validation loss 0.4013
Epoch 1000, Training loss 0.0298, Validation loss 0.0253
Epoch 2000, Training loss 0.0280, Validation loss 0.0241
Epoch 3000, Training loss 0.0274, Validation loss 0.0238
Epoch 4000, Training loss 0.0271, Validation loss 0.0236
Epoch 5000, Training loss 0.0270, Validation loss 0.0235

Parameter containing:
tensor([[ 0.0398,  0.0670, -0.0053, -0.0172,  0.0045, -0.0020, -0.0019, -0.0037,
         -0.0003, -0.0130, -0.0027]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([0.2356], dtype=torch.float64, requires_grad=True)


In [ ]:
training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = linear_model,
    loss_fn = nn.MSELoss(), # <1>
    t_u_train = X_train,
    t_u_val = X_val,
    t_c_train = y_train,
    t_c_val = y_val)

print()
print(linear_model.weight)
print(linear_model.bias)

Neural Network with Single Layer

In [ ]:
from collections import OrderedDict

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(11, 32)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(32, 1))
])).double()

seq_model

In [ ]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

In [ ]:
seq_model.output_linear.bias

In [ ]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3) # <1>

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = X_train,
    t_u_val = X_val,
    t_c_train = y_train,
    t_c_val = y_val)

#print('output', seq_model(X_val))
#print('answer', y_val)
#print('hidden', seq_model.hidden_linear.weight.grad)

Epoch 500, Training loss 0.0271, Validation loss 0.0238
Epoch 1000, Training loss 0.0271, Validation loss 0.0238
Epoch 1500, Training loss 0.0271, Validation loss 0.0238
Epoch 2000, Training loss 0.0270, Validation loss 0.0238
Epoch 2500, Training loss 0.0270, Validation loss 0.0237
Epoch 3000, Training loss 0.0270, Validation loss 0.0237
Epoch 3500, Training loss 0.0270, Validation loss 0.0237
Epoch 4000, Training loss 0.0270, Validation loss 0.0237
Epoch 4500, Training loss 0.0270, Validation loss 0.0237
Epoch 5000, Training loss 0.0270, Validation loss 0.0237


Neural Network with Multiple Hidden Layers

In [ ]:
seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(11, 32)),
    ('hidden_activation', nn.Tanh()),
    ('hidden_layer2', nn.Linear(32, 64)),
    ('hidden_activation2', nn.Tanh()),
    ('hidden_layer3', nn.Linear(64, 16)),
    ('hidden_activation3', nn.Tanh()),
    ('output_linear', nn.Linear(16, 1))
])).double()

seq_model

Sequential(
  (hidden_linear): Linear(in_features=11, out_features=32, bias=True)
  (hidden_activation): Tanh()
  (hidden_layer2): Linear(in_features=32, out_features=64, bias=True)
  (hidden_activation2): Tanh()
  (hidden_layer3): Linear(in_features=64, out_features=16, bias=True)
  (hidden_activation3): Tanh()
  (output_linear): Linear(in_features=16, out_features=1, bias=True)
)

In [ ]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3) # <1>

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = X_train,
    t_u_val = X_val,
    t_c_train = y_train,
    t_c_val = y_val)

Epoch 500, Training loss 0.0270, Validation loss 0.0237
Epoch 1000, Training loss 0.0270, Validation loss 0.0237
Epoch 1500, Training loss 0.0270, Validation loss 0.0237
Epoch 2000, Training loss 0.0270, Validation loss 0.0237
Epoch 2500, Training loss 0.0270, Validation loss 0.0237
Epoch 3000, Training loss 0.0270, Validation loss 0.0237
Epoch 3500, Training loss 0.0270, Validation loss 0.0237
Epoch 4000, Training loss 0.0270, Validation loss 0.0237
Epoch 4500, Training loss 0.0270, Validation loss 0.0236
Epoch 5000, Training loss 0.0270, Validation loss 0.0236
